# Measuring energy and energy variance of an Ising Hamiltonian with classical shadows

Related Paper [Huang et al, Nat Phys 2020](https://doi.org/10.1038/s41567-020-0932-7)

In [1]:
using RandomMeas
using Statistics

We first construct the Hamiltonian of the transverse field Ising model with N qubits. 

In [2]:
N = 6 # Number of qubits

ξ = siteinds("Qubit", N) # ITensor site indices for classical shadows
# Define the Hamiltonian
J = -1. # Ising interaction strength
B = 1. # Transverse field strength
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising XX interactions
  ampo .+= J , "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field Z
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ) # Hamiltonian as MPO
H2 = apply(H,H); # H^2 for later use

## Ground state 

Next, we compute the ground state $\psi_G$ of the transverse field Ising model with DMRG.

In [3]:
# Define the DMRG parameters 
dmrg_iter = 6      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = random_mps(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100, 500)
cutoff!(sweeps, dmrg_cutoff)

# Run DMRG to get the ground state
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψG = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-7.215340017432208  maxlinkdim=4 maxerr=1.29E-16 time=8.645
After sweep 2 energy=-7.296138053303525  maxlinkdim=8 maxerr=1.77E-16 time=0.005
After sweep 3 energy=-7.296229809762754  maxlinkdim=6 maxerr=2.72E-11 time=0.004
After sweep 4 energy=-7.296229810529878  maxlinkdim=6 maxerr=2.53E-12 time=0.003
After sweep 5 energy=-7.296229810530153  maxlinkdim=6 maxerr=2.50E-12 time=0.002
After sweep 6 energy=-7.296229810530149  maxlinkdim=6 maxerr=2.50E-12 time=0.005

Ground state energy:  -7.296229810530149


We compute its energy and variance.

In [4]:
E_G = real(inner(ψG', H,ψG))
println("Ground state energy: ",E_G)
E2_G = real(inner(ψG', H2,ψG))
println("Expectation value of H2: ",E2_G)
Var_E_G = E2_G-E_G^2
println("Variance: ",Var_E_G)

Ground state energy: -7.296229810530152
Expectation value of H2: 53.23496944839764
Variance: 3.2878944011827116e-10


## Memory efficient simulation and post-processing 

We simulate an experiment (memory-efficient) by sampling directly from with groundstate MPS.

In [16]:
NU = 1000 # Number of measurement settings
NM = 50 # Number of measurements per setting
@time measurement_group = MeasurementGroup(ψG,NU,NM;mode=TensorNetwork,progress_bar=true);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


 10.978490 seconds (63.98 M allocations: 6.745 GiB, 8.70% gc time, 51.19% compilation time)


We now construct (memory efficient) factorized classical shadows.

In [17]:
@time factorized_shadows = get_factorized_shadows(measurement_group);

  4.860571 seconds (36.93 M allocations: 4.681 GiB, 12.72% gc time, 34.32% compilation time)


We use them to estimate the groundstate energy and its variance.

In [18]:
Es_m,Es_ste = get_expect_shadow(H,factorized_shadows,compute_sem=true)
E2s_m,E2s_ste = get_expect_shadow(H2,factorized_shadows,compute_sem=true)
Es_m = real(Es_m)
E2s_m = real(E2s_m)
Evar = E2s_m .- Es_m.^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy  ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2  ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.

Estimated energy  -7.31±0.03
Estimated expectation value of H^2  53.1±0.4
Estimated energy variance -0.3±-0.2


## Faster memory inefficient simulation and post-processing

We simulate an experiment by constructing first a dense probability tensor and use this to sample projective measurements. This works only for small systems $N\lesssim 10$, but provides here an advantage in terms of speed.

In [19]:
NU = 1000 # Number of measurement settings
NM = 50 # Number of measurements per setting
@time measurement_group = MeasurementGroup(ψG,NU,NM;mode=Dense,progress_bar=true);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


 13.596788 seconds (38.74 M allocations: 3.166 GiB, 3.62% gc time, 91.49% compilation time)


We construct $N_U$ dense classical shadows (i.e.\ matrices of dimension 2^N).

In [20]:
ITensors.disable_warn_order()

@time dense_shadows = get_dense_shadows(measurement_group);

 26.304368 seconds (77.41 M allocations: 5.133 GiB, 2.75% gc time, 97.69% compilation time)


We estimate energy and Hamiltonian variance with dense classical shadows. 

In [21]:
Es_m,Es_ste = get_expect_shadow(H,dense_shadows,compute_sem=true)
E2s_m,E2s_ste = get_expect_shadow(H2,dense_shadows,compute_sem=true)
Es_m = real(Es_m)
E2s_m = real(E2s_m)
Evar = E2s_m .- Es_m.^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy  ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2  ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.

Estimated energy  -7.27±0.08
Estimated expectation value of H^2  53.1±0.9
Estimated energy variance 0.3±0.4


# Output state of quantum circuit

We construct a state as output of a quantum circuit.

In [22]:
η = 3 # Number of layer blocks
m = 3 # Number of layers per block
# Build the gate structure
circuit = ITensor[]
for d in 1:η
    xx_layer = [op("Rxx", ξ[j], ξ[j+1], ϕ=1*d/η/m) for j in 1:(N - 1)] 
    z_layer = [op("Rz", ξ[j], θ=2*B/m) for j in 1:N] 
    for t in 1:m
        append!(circuit, xx_layer)
        append!(circuit, z_layer)
    end
end
ψ0 = MPS(ξ,["Up" for n in 1:N]);
ψt = apply(circuit,ψ0; cutoff=1E-8)
normalize!(ψt);


We compute its energy and variance.

In [23]:
Et = real(inner(ψt', H,ψt)) # Energy of ψt 
println("final energy ",Et)
Var_Et = real(inner(ψt', H2,ψt))-Et^2 # Energy variance of ψt
println("final energy variance ",Var_Et)

final energy -6.511387773163446
final energy variance 3.6196819415243695


In [24]:
NU = 1000 # Number of random unitaries
NM = 50 # Number of measurements per unitary
measurement_group = MeasurementGroup(ψt,NU,NM;mode=TensorNetwork,progress_bar=true);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


In [25]:
factorized_shadows = get_factorized_shadows(measurement_group);

In [26]:
Es_m,Es_ste = get_expect_shadow(H,factorized_shadows,compute_sem=true)
E2s_m,E2s_ste = get_expect_shadow(H2,factorized_shadows,compute_sem=true)
Es_m = real(Es_m)
E2s_m = real(E2s_m)
Evar = E2s_m .- Es_m.^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy of the circuit generated state ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2 of the circuit generated state ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance of the circuit generated state ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.

Estimated energy of the circuit generated state -6.58±0.04
Estimated expectation value of H^2 of the circuit generated state 46.2±0.4
Estimated energy variance of the circuit generated state 2.9±2.0
